<a href="https://colab.research.google.com/github/jiiiisoo/ICT/blob/main/helmet_detection/helmet_detection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import xml.etree.ElementTree as et
from sklearn.model_selection import train_test_split
import random

import cv2
import imutils
import matplotlib.pyplot as plt

In [3]:
#unzip image data file

!unzip -uq "/content/drive/MyDrive/deeplearning/final_project/archive.zip" -d "/content/drive/MyDrive/deeplearning/final_project"

In [2]:
X=[]
Y=[]

#image 불러오기
image_list = os.listdir("/content/drive/MyDrive/deeplearning/final_project/images")
label_list = os.listdir("/content/drive/MyDrive/deeplearning/final_project/annotations")

print(len(image_list), len(label_list))

764 764


In [3]:
#label: With Helmet=1, Without Helmet=0
width=128
height=128

image_set=[]
for img in image_list:
    label=[]
    classnum = []
    xtree = et.parse('/content/drive/MyDrive/deeplearning/final_project/annotations/' + img.split('.')[0] + '.xml')
    #size = xtree.find('size')
    cv2_image = cv2.imread(os.path.join('/content/drive/MyDrive/deeplearning/final_project/images', img),cv2.IMREAD_COLOR)
    resized = cv2.resize(cv2_image, (width,height)) #보통 이미지 축소에는 INTER_AREA 사용

    for e in xtree.findall('object'):
      name = e.find('name').text
      label.append({'name': name,})
      if name == 'Without Helmet':
        label_num=0
      elif name == 'With Helmet':
        label_num=1
      #name이 둘 다 아닌 경우가 있을 수 있으므로
      else:
        label_num=2
    image_set.append((resized, label_num))
    
print(len(image_set))

764


In [4]:
#Split data into train, test dataset

random.shuffle(image_set)

train_set, test_set = train_test_split(image_set, test_size=0.2, random_state=1)

print(len(train_set), len(test_set))

611 153


In [5]:
# image, label 분리
train_image=[]
train_label=[]
test_image=[]
test_label=[]

for image, label in train_set:
  train_image.append(image)
  train_label.append(label)

for image, label in test_set:
  test_image.append(image)
  test_label.append(label)

#차원 조정 ->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>왱?!
train_image=np.array(train_image).reshape(-1,width,height,1)
test_image=np.array(test_image).reshape(-1,width,height,1)

# Normalize pixel values to be between 0 and 1
train_image=train_image/255.0

#image 증강 통해 overfitting 방지하고자 imagedatagenerator로 이미지 변형
datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True)

In [7]:
train_datagen = datagen.flow(train_image, train_label, batch_size=16)
test_datagen = datagen.flow(test_image, test_label, batch_size=16)

ValueError: ignored

In [20]:
# model 구축

model=models.Sequential()
model.add(layers.Conv2D(128,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 128)     3584      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 63, 128)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        73792     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 30, 30, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)       

reference  
https://hwiyong.tistory.com/48  
https://keras.io/ko/preprocessing/image/  
https://yunwoong.tistory.com/127